In [1]:
%matplotlib qt

In [2]:
import os
import pims
import xarray as xr
import matplotlib
import matplotlib.pylab as plt

import numpy as np
import holopy as hp
from holopy.core.process import bg_correct
from holopy.propagation import ps_propagate
from scipy.ndimage import center_of_mass


/home/scholz_la/.conda/envs/holography/lib/python3.9/site-packages/pims/image_reader.py:26: RuntimeWarning: PIMS image_reader.py could not find scikit-image. Falling back to matplotlib's imread(), which uses floats instead of integers. This may break your scripts. 
(To ignore this warning, include the line "warnings.simplefilter("ignore", RuntimeWarning)" in your script.)
  warnings.warn(RuntimeWarning(ski_preferred))


In [30]:
from holopy.core.io import save_images

In [3]:
from holopy import check_display
check_display()
plt.show()

virtual void QEventDispatcherUNIX::registerSocketNotifier(QSocketNotifier*): Multiple socket notifiers for same socket 11 and type Read


# Holography
## Setup - load data

In [34]:
imagepath = "/media/nif9202/Luis/Holography/Recording Single frame/worm2.bmp"
bgpath = "/media/nif9202/Luis/Holography/Recording Single frame/BackgroundWorm2.bmp"

#image = pims.open(imagepath)
raw_holo = hp.load_image(imagepath)
hp.show(raw_holo)

/home/scholz_la/.conda/envs/holography/lib/python3.9/site-packages/holopy/core/metadata.py:344: UserWarning: No pixel spacing provided. Setting spacing to 1, but anysubsequent calculations will be wrong.
  warn("No pixel spacing provided. Setting spacing to 1, but any"


## Background removal

In [35]:
bg = hp.load_image(bgpath)
holo = bg_correct(raw_holo, bg)
hp.show(holo)

# for multiple background we can load a series
#bgpath = get_example_data_path(['bg01.jpg', 'bg02.jpg', 'bg03.jpg'])
#bg = hp.core.io.load_average(bgpath, refimg = raw_holo)

/home/scholz_la/.conda/envs/holography/lib/python3.9/site-packages/holopy/core/metadata.py:344: UserWarning: No pixel spacing provided. Setting spacing to 1, but anysubsequent calculations will be wrong.
  warn("No pixel spacing provided. Setting spacing to 1, but any"


## specify experimental details
Spacing and wavelength must be given in the same units - micrometers in the example above. Holopy has no built-in length scale and requires only that you be consistent. For example, we could have specified both parameters in terms of nanometers or meters instead.

In [36]:
# spacing
# we have 6x magnification -> 2.34um/6
spacing = 2.34/6
# wavelength incident light
lambda_ill = 0.590
# direction of polarization 
polarization = (1,0)
# refractive index of medium
medium_index = 1.33
holo = hp.core.update_metadata(holo, medium_index=medium_index,
                               illum_wavelen=lambda_ill, illum_polarization=polarization)

## Reconstruct a single image

In [37]:
zstack = np.linspace(-5000, -10000, 21)
rec_vol = hp.propagate(holo, zstack,cfsp = 3)
hp.show(rec_vol)

/home/scholz_la/.conda/envs/holography/lib/python3.9/site-packages/holopy/core/io/vis.py:264: UserWarning: Image contains complex values. Taking image magnitude.
  warn("Image contains complex values. Taking image magnitude.")


## repeat for a whole movie

In [60]:
moviepath = "/media/nif9202/Luis/Holography/Recording Movie Worm/"
bgpath = "/media/nif9202/Luis/Holography/Recording Movie Worm/Background.bmp"


In [17]:
# spacing
# we have 6x magnification -> 2.34um/6
spacing = 2.34/6
# wavelength incident light
lambda_ill = 0.590
# direction of polarization 
polarization = (1,0)
# refractive index of medium
medium_index = 1.33

In [56]:
def run_holography(fn, bg,zstack, **pars):
    """run all steps of reconstruction."""
    raw_holo = hp.load_image(fn, spacing = spacing)
    holo = bg_correct(raw_holo, bg)
    holo = hp.core.update_metadata(holo, medium_index=medium_index,
                           illum_wavelen=lambda_ill, illum_polarization=polarization)
    return hp.propagate(holo, zstack)

In [65]:
%%time
outfile = 'Holo_{}_{}.tiff'
zstack = np.linspace(-2000, 2000, 10)
pars = {'medium_index':medium_index,
        'illum_wavelen':lambda_ill, 
        'illum_polarization':polarization,
       'spacing':spacing}
bg = hp.load_image(bgpath, spacing = pars['spacing'])

for fn in os.listdir(moviepath)[:6]:
    if fn.endswith('tiff') and 'Basler' in fn:
        # create volume reconstruction
        idx = fn.split('_')[-1].split('.')[0]
        impath = os.path.join(moviepath, fn)
        rec = run_holography(impath, bg,zstack, **pars)
        # save images
        filenames = [outfile.format(k,idx) for k in range(len(zstack))]
        save_images(filenames,np.moveaxis(rec.values, -1,0))
        
#         movie.append(rec)
# rec_movie = xr.concat(movie, 't')

/home/scholz_la/.conda/envs/holography/lib/python3.9/site-packages/holopy/core/io/vis.py:264: UserWarning: Image contains complex values. Taking image magnitude.
  warn("Image contains complex values. Taking image magnitude.")
/home/scholz_la/.conda/envs/holography/lib/python3.9/site-packages/holopy/core/io/vis.py:264: UserWarning: Image contains complex values. Taking image magnitude.
  warn("Image contains complex values. Taking image magnitude.")
/home/scholz_la/.conda/envs/holography/lib/python3.9/site-packages/holopy/core/io/vis.py:264: UserWarning: Image contains complex values. Taking image magnitude.
  warn("Image contains complex values. Taking image magnitude.")
/home/scholz_la/.conda/envs/holography/lib/python3.9/site-packages/holopy/core/io/vis.py:264: UserWarning: Image contains complex values. Taking image magnitude.
  warn("Image contains complex values. Taking image magnitude.")
/home/scholz_la/.conda/envs/holography/lib/python3.9/site-packages/holopy/core/io/vis.py:264

/home/scholz_la/.conda/envs/holography/lib/python3.9/site-packages/holopy/core/io/vis.py:264: UserWarning: Image contains complex values. Taking image magnitude.
  warn("Image contains complex values. Taking image magnitude.")
/home/scholz_la/.conda/envs/holography/lib/python3.9/site-packages/holopy/core/io/vis.py:264: UserWarning: Image contains complex values. Taking image magnitude.
  warn("Image contains complex values. Taking image magnitude.")
/home/scholz_la/.conda/envs/holography/lib/python3.9/site-packages/holopy/core/io/vis.py:264: UserWarning: Image contains complex values. Taking image magnitude.
  warn("Image contains complex values. Taking image magnitude.")


CPU times: user 41.7 s, sys: 7.62 s, total: 49.3 s
Wall time: 53.3 s


In [64]:
zstack = np.linspace(0, 5000, 11)
rec_vol =run_holography(impath, bg,zstack, **pars)
hp.show(rec_vol)

/home/scholz_la/.conda/envs/holography/lib/python3.9/site-packages/holopy/core/io/vis.py:264: UserWarning: Image contains complex values. Taking image magnitude.
  warn("Image contains complex values. Taking image magnitude.")


pixels: x = 1161, y = 1613, z = 0, units: x = 4.5e+02, y = 6.3e+02, z = 0.0e+00,


virtual void QEventDispatcherUNIX::registerSocketNotifier(QSocketNotifier*): Multiple socket notifiers for same socket 11 and type Read


## Postprocessing

In [37]:
plt.imshow(np.max(np.abs(rec_vol.values), axis=-1))

virtual void QEventDispatcherUNIX::registerSocketNotifier(QSocketNotifier*): Multiple socket notifiers for same socket 11 and type Read


## Loading and saving data

In [ ]:
outfile = 'test'
hp.save(outfile, holo)
holo = hp.load(outfile)
hp.save_image(outfile, holo)

In [ ]:
outfile = 'test'
hp.save(outfile,movie)
holo = hp.load(outfile)

In [ ]:
pwd